### Create a dataframe containing info for all movies

In [ ]:
import numpy as np
import requests
import re
import pandas as pd
import os
import matplotlib.pyplot as plt
from google.cloud import storage
from bs4 import BeautifulSoup
from google.cloud import storage


In [ ]:
# retrieve the datasets in a .csv file. RUN ONCE to update data
# %run scraping_imdb_movies.ipynb

In [ ]:
# combine files into a single dataframe
# movies_df = get_all_movies(csv_list)   
# movies_csv = movies_df.to_csv('movies.csv', index=False)

In [ ]:
# store dataset in gcs. RUN ONCE
# client = storage.Client()
# bucket = client.get_bucket('ezhou1-bucket1')
# bucket.blob('imdb/movies.csv').upload_from_string(movies_df.to_csv(index=False), 'movies.csv')

In [ ]:
# retrieve unprocessed data from gcs
movies_df = pd.read_csv('gs://bucket/movies.csv')

## Data Understanding and Preparation  
- Describing the dataframe
- Dropping irrelevant columns/rows  
- Renaming columns  
- Feature creation

In [ ]:
movies_df.head(5)

,Unnamed: 0,title,year,certificate,time,genre,rating,metascore,simple_desc,directors,stars,votes,gross
0,0,Spider-Man: Across the Spider-Verse,(2023),PG,140 min,"Animation, Action, Adventure",9.1,86,"Miles Morales catapults across the Multiverse,...","['Joaquim Dos Santos', 'Kemp Powers', 'Justin ...","['Shameik Moore', 'Hailee Steinfeld', 'Brian T...","79,742",-
1,1,The Dark Knight,(2008),PG-13,152 min,"Action, Crime, Drama",9.0,84,When the menace known as the Joker wreaks havo...,['Christopher Nolan'],"['Christian Bale', 'Heath Ledger', 'Aaron Eckh...","2,721,730",$534.86M
2,2,The Lord of the Rings: The Return of the King,(2003),PG-13,201 min,"Action, Adventure, Drama",9.0,94,Gandalf and Aragorn lead the World of Men agai...,['Peter Jackson'],"['Elijah Wood', 'Viggo Mortensen', 'Ian McKell...","1,889,031",$377.85M
3,3,Inception,(2010),PG-13,148 min,"Action, Adventure, Sci-Fi",8.8,74,A thief who steals corporate secrets through t...,['Christopher Nolan'],"['Leonardo DiCaprio', 'Joseph Gordon-Levitt', ...","2,415,873",$292.58M
4,4,The Lord of the Rings: The Fellowship of the Ring,(2001),PG-13,178 min,"Action, Adventure, Drama",8.8,92,A meek Hobbit from the Shire and eight compani...,['Peter Jackson'],"['Elijah Wood', 'Ian McKellen', 'Orlando Bloom...","1,917,638",$315.54M


In [ ]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4355 entries, 0 to 4354
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   4355 non-null   int64  
 1   title        4355 non-null   object 
 2   year         4355 non-null   object 
 3   certificate  4355 non-null   object 
 4   time         4355 non-null   object 
 5   genre        4355 non-null   object 
 6   rating       4355 non-null   float64
 7   metascore    4355 non-null   object 
 8   simple_desc  4355 non-null   object 
 9   directors    4355 non-null   object 
 10  stars        4355 non-null   object 
 11  votes        4355 non-null   object 
 12  gross        4355 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 442.4+ KB


In [ ]:
movies_df.columns
df=movies_df.drop_duplicates().copy() # drops all 42 duplicate rows

In [ ]:
df.shape

(4315, 13)

In [ ]:
df=df.rename(columns={"Unnamed: 0": "id", 'time':'time_min', 'gross':'gross_mil_usd', 'stars':'actors'}).copy()
df['time_min']=df['time_min'].str.replace('min', '',regex=False)
df['gross_mil_usd']=df['gross_mil_usd'].str.replace('\$|M', '')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df['year']=df['year'].str.extract('(\d+)').astype(int)

In [ ]:
df['time_min']=df['time_min'].str.extract('(\d+)').astype(float)

In [ ]:
df['gross_mil_usd']=df['gross_mil_usd'].str.extract('(\d+\.\d+)').astype(float)

In [ ]:
df['metascore']=df['metascore'].str.extract('(\d+)').astype(float)

In [ ]:
df['votes'] = df['votes'].str.replace(',', '').astype(float)

In [144]:
idx = np.arange(len(df['title']))
df['id'] = idx

In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4315 entries, 0 to 4354
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             4315 non-null   int64  
 1   title          4315 non-null   object 
 2   year           4315 non-null   int64  
 3   certificate    4315 non-null   object 
 4   time_min       4315 non-null   float64
 5   genre          4315 non-null   object 
 6   rating         4315 non-null   float64
 7   metascore      3725 non-null   float64
 8   simple_desc    4315 non-null   object 
 9   directors      4315 non-null   object 
 10  actors         4315 non-null   object 
 11  votes          4315 non-null   float64
 12  gross_mil_usd  3498 non-null   float64
dtypes: float64(5), int64(2), object(6)
memory usage: 601.0+ KB


In [148]:
# THE FOLLOWING IS PREPARATION FOR big_query.ipynb
# store dataset in gcs. RUN ONCE
client = storage.Client()
bucket = client.get_bucket('ezhou1-bucket1')    
bucket.blob('imdb/movies_cleaned.csv').upload_from_string(df.to_csv(index=False), 'movies_cleaned.csv')

Visualize data

In [ ]:
df['directors'] = df['directors'].apply(eval) #convert list of type String to type List
df['actors'] = df['actors'].apply(eval) #convert list of type String to type List

# Get unique items & counts
actor= df['actors'].explode().value_counts().index.tolist()
frequency = df['actors'].explode().value_counts().values
df2 = pd.DataFrame({
    'actor': actor[:20],
    'frequency' : frequency[:20]
})

# Create plot
plt.barh(y=df2.actor, width=df2.frequency)
plt.xlabel('Number of Movies Starred In')
plt.title('Number of movies per actor (top 20)')
plt.gca().invert_yaxis()
plt.show()